# Это ДЗ ещё не готово

# Урок 7. Модель Transformer.

Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше<br>
Поэкспериментировать с переводом написать по вашим наблюдениям где он ошибается, попробовать изменить архитектуру